1. 引用我自己寫的 helper function: query_from_db可以直接從DB獲取資料>
2. Import 會用到的 library
3. 定義了python request 連線的 header

In [1]:
# Import the pre-implement db query function

import os, sys
parent_dir = os.path.dirname(os.getcwd())
#os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(parent_dir)
from db_func import query_from_db
import html
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time

headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}

##### 將資料print出來的function，方便debug。

In [2]:
def print_util(urls, processor):
    for url in urls:
        try:
            print()
            temp = processor(url[0])
            print('News Url: \n{}\n'.format(url[0]))
            #print("Title: {}".format(temp['news_title']))
            #print("News: {}".format(temp['news']))
            for key in temp.keys():
                print('{}: {}'.format(key.capitalize(), temp[key]))
            print()
            print()
        except Exception as e:
            print()
            print(e)
            print('ERROR URL:')
            print(url)
            print()

### 選出10個在DB之中有最多資料的新聞來源：
```sql
SELECT news_source,COUNT(*) 
FROM news_db.news_rss_feeds
GROUP BY news_source
ORDER BY 2 DESC
LIMIT 11;
```

|  news_source    | COUNT(*) |
|------------------|----------|
| MSN              | 14204    |
| 自由時報         | 4997     |
| 大紀元           | 4394     |
| 經濟日報         | 3589     |
| ~新浪台灣新聞中心~ | ~3323~     |
| PCHOME           | 3213     |
| ETtoday          | 2585     |
| 新頭殼要聞       | 2117     |
| Rti 中央廣播電臺 | 2010     |
| Yahoo!奇摩股市   | 1976     |
| 公視新聞 |1796|  
最後因為新浪新聞有很多url是無效的，因此沒有放入 DB

### 1. 隨機選出50個Yahoo的新聞連結：
在開發時我是確保50個新聞連結都通過，但為了怕notebook篇幅太長，我改成五個來demo
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Yahoo!奇摩股市' ORDER BY RAND()
LIMIT 50;
```

In [3]:
random_yahoo_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source LIKE '%Yahoo%' ORDER BY RAND()
LIMIT 100; """

yahoo_test_urls = query_from_db(random_yahoo_sql)

In [23]:
def yahoo_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    stop_footers = set(['全球疫情大流行', '今日最夯新聞流量前3名', '更多相關新聞', '更多 NOWnews 今日新聞 報導 ', '看更多 CTWANT 文章', '更多上報內容：', '更多 TVBS 報導', '《更多匯流新聞網CNEWS報導》', '更多社會相關新聞', '更多東森新聞報導', '原始連結', '更多鉅亨報導', '更多國際相關新聞', '***本文所提之內容僅供參考，不具投資建議，投資前應審慎評估風險，且自負盈虧***', '延伸閱讀：', '更多華視新聞報導', '更多匯流新聞網報導：', '【延伸閱讀】', '更多民視新聞報導', '更多財經相關新聞', '※【NOWnews 今日新聞】提醒您：', '更多健康2.0報導', '更多三立新聞網報導', '超人氣', '《TVBS》提醒您：', '《更多CNEWS匯流新聞網報導》', '更多新聞推薦', '更多新頭殼報導', '更多Heho健康網文章'])
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = html.unescape(title_category[0])
        if len(title_category) > 1:
            res_dict['news_category'] = html.unescape(title_category[1])
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = html.unescape(keywords_tag['content'])

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = html.unescape(description_tag['content'])

    time_tag = soup.find('time')
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")
            db_time_format = '%Y-%m-%d %H:%M:%S'
            res_time = d1.strftime(db_time_format)
            res_dict['published_time'] = res_time

            d2 = d1 + datetime.timedelta(hours=8)
            d2 = d2.date()
            db_date_format = '%Y-%m-%d'
            res_date = d2.strftime(db_date_format)
            res_dict['published_date'] = res_date
        except Exception as e:
            logging.error(e)
            print(e)
            pass

    article_body_tag = soup.find('article', attrs = {'itemprop':'articleBody'})
    caas_body_tag = soup.find('div', attrs = {'class': 'caas-body'})
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for index, p in enumerate(p_tags):
                if p.get('content'):
                    if any([html.unescape(p.get_text()).startswith(stop_footer) for stop_footer in stop_footers]):
                        break
                    temp_content.append(html.unescape(p.get_text().strip()))
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif caas_body_tag:
        p_tags = caas_body_tag.find_all('p')
        a_tags = caas_body_tag.find_all('a')

        if p_tags:
            for index, p in enumerate(p_tags):
                if any([html.unescape(p.get_text()).startswith(stop_footer) for stop_footer in stop_footers]):
                    break
                temp_content.append(html.unescape(p.get_text()).strip())
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    if temp_content:    
        # Capture the description start with 公告 to content
        if title_category[0][:4] == '【公告】':
            prefix = title_category[0]
        else:
            prefix = ''
        content = prefix + '\n'.join(temp_content)#.replace('。 ', '。\n')
        res_dict['news'] = html.unescape(content)
        return res_dict
    else:
        print('Yahoo url: {} did not process properly'.format(url))
        #content_parser_1.logger.error('Yahoo url: {} did not process properly'.format(url))
        #content_parser.errors['process_empty_content_(rss_id)'].append([rss_id, url])
        return

In [24]:
yahoo_test_urls[:10]

,news_url
0,https://tw.news.yahoo.com/%E5%A7%9A%E9%9B%A8%E...
1,https://tw.news.yahoo.com/%E4%B8%8D%E7%94%A9%E...
2,https://tw.news.yahoo.com/%E5%85%A8%E6%96%87-%...
3,https://tw.news.yahoo.com/%E4%B8%AD%E7%A3%8A%E...
4,https://tw.news.yahoo.com/%E5%8F%B0%E8%82%A1%E...
5,https://tw.news.yahoo.com/%E5%A4%A7%E8%B0%B7%E...
6,https://tw.stock.yahoo.com/news/%E7%BE%8E%E8%8...
7,https://tw.news.yahoo.com/%E7%B2%BE%E5%93%81%E...
8,https://tw.news.yahoo.com/%E5%BF%AB%E8%A8%8A-%...
9,https://tw.stock.yahoo.com/news/%E5%8D%8A%E5%B...


In [25]:
print('Yahoo 新聞：')
print_util(yahoo_test_urls[:10].values, yahoo_content_processor)

Yahoo 新聞：

News Url: 
https://tw.news.yahoo.com/%E5%A7%9A%E9%9B%A8%E7%BF%94%E4%BB%A3%E6%89%93%E9%80%86%E8%BD%89-%E7%8C%9B%E7%8D%85%E5%92%AC%E5%91%B3%E5%85%A8%E9%BE%8D%E9%96%8B%E8%83%A1-120430347.html

News_title: 姚雨翔代打逆轉 猛獅咬味全龍開胡
News_keywords: 王維中, 安打, 姚雨翔
News_description: 味全龍隊今天回歸首戰在台南球場遭遇統一獅隊，龍隊狀元王維中先發主投4.2局失3分，被打5安打，3保送，吞下敗投，而獅隊靠著姚雨翔代打敲出2分打點逆轉安打，終場以3:2擊敗味全龍。味全打線由3局上曾傳昇...
Published_time: 2021-03-14 12:04:30
Published_date: 2021-03-14
News_related_url: ['https://www.nownews.com/news/5211579?from=Yahoo&utm_source=NaturalLink&utm_medium=Yahoo&utm_campaign=20210315', 'https://www.nownews.com/news/5211572?from=Yahoo&utm_source=NaturalLink&utm_medium=Yahoo&utm_campaign=20210315', 'https://www.nownews.com/news/5211546?from=Yahoo&utm_source=NaturalLink&utm_medium=Yahoo&utm_campaign=20210315']
News_related_url_desc: ['中職／味全龍回歸 陳品捷首安、首轟都包了', '中職／王維中撐不過5局 淪為敗投選人', '被罰站10分鐘！外送遇「奇葩奧客」狂刁難 結局網全氣瘋']
News: 味全龍隊今天回歸首戰在台南球場遭遇統一獅隊，龍隊狀元王維中先發主投4.2局失3分，被打5安打，3保送，吞下敗投，而獅隊靠著姚雨翔代打敲出2分打點逆轉安打，終場以3:2擊敗味全龍。
味全

### 2. 隨機選出50個 MSN 的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 50;
```

In [26]:
random_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 100; """
msn_test_urls = query_from_db(random_msn_sql)

In [28]:

def msn_content_processor(url):
    res_dict = {}
    prefix = ['相關報導', '※', '＊', '更多三立新聞網報導', '延伸閱讀', '資料來源', '更多中時電子報精彩報導', '看了這篇文章的人，也']
    pattern = re.compile(r'^{}'.format('|'.join(prefix)))
    
    if url.startswith('https://www.msn.com/zh-tw/video/'):
        return
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        res_dict['news_title'] = html.unescape(title_tag.text.strip())
        
    
    category_tag = soup.find('div', attrs = {'class':'logowrapper'})
    if category_tag:
        if len(category_tag.get_text().strip().split('\n')) == 2:
            _, category = category_tag.get_text().strip().split('\n')
            res_dict['news_category'] = html.unescape(category)
            
    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = html.unescape(description_tag['content'])

    time_tag = soup.find('div', attrs = {'class': 'timeinfo-txt'})
    if time_tag:
        date_time_tag = time_tag.find('time')
        try:
            d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.000Z") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('MSN date error, url: {}'.format(url))
    
    article_body_tag = soup.find('section', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'content'})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    elif article_body_tag_2:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
        
    if len(temp_content):
        content = '\n'.join(temp_content)
        res_dict['news'] = html.unescape(content)

    if not res_dict or 'news' not in res_dict: 
        #content_parser.logger.error('MSN url: {} did not process properly'.format(url))
        #content_parser.errors['process_empty_content_(rss_id)'].append([rss_id, url])
        print('MSN url: {} did not process properly'.format(url))
        return 
    return res_dict

In [29]:
print('MSN 新聞：')
print_util([['https://www.msn.com/zh-tw/money/topstories/%E5%8F%B0%E7%A9%8D%E9%9B%BB%E6%89%BF%E8%AB%BE%E5%8A%A0%E9%80%9F%E4%BE%9B%E7%B5%A6-%E5%88%B0%E5%BA%95%E8%BB%8A%E7%94%A8%E6%99%B6%E7%89%87%E7%BC%BA%E8%B2%A8%E7%BC%BA%E4%BB%80%E9%BA%BC-%E8%AA%BF%E7%A0%94-%E9%80%993%E7%94%A2%E7%B7%9A%E6%9C%80%E7%B7%8A%E7%BC%BA/ar-BB1d8oqq?li=BBqj0iS&amp;srcref=rss%27']], msn_content_processor)

MSN 新聞：

News Url: 
https://www.msn.com/zh-tw/money/topstories/%E5%8F%B0%E7%A9%8D%E9%9B%BB%E6%89%BF%E8%AB%BE%E5%8A%A0%E9%80%9F%E4%BE%9B%E7%B5%A6-%E5%88%B0%E5%BA%95%E8%BB%8A%E7%94%A8%E6%99%B6%E7%89%87%E7%BC%BA%E8%B2%A8%E7%BC%BA%E4%BB%80%E9%BA%BC-%E8%AA%BF%E7%A0%94-%E9%80%993%E7%94%A2%E7%B7%9A%E6%9C%80%E7%B7%8A%E7%BC%BA/ar-BB1d8oqq?li=BBqj0iS&amp;srcref=rss%27

News_title: 台積電承諾加速供給！到底車用晶片缺貨缺什麼？調研：這3產線最緊缺
News_category: 財經
News_description: 經濟部針對全球車用晶片缺貨問題，邀請台積電、聯電、力積電、世界先進等國內主要代工廠代表一起商討。
Published_time: 2021-01-27 21:38:00
Published_date: 2021-01-28
News_related_url: []
News_related_url_desc: []
News: 編按（2021.1.28 13:30更新）：針對車用晶片缺貨問題，TrendForce報告指出，自2018年起車市逐步疲軟，加上2020年受到疫情嚴重衝擊，使主要模組廠的備貨動能明顯不足。然而，2021年全球汽車市場正在復甦，預估整車銷售量將自去年的7,700萬輛回升至8,400萬輛，同時汽車在自動化、智慧化和電動化發展下，對於各種半導體元件的用量將大幅上升，然先前因車市需求疲軟導致車廠備貨量偏低，長短料的現象已嚴重影響車廠稼動率與終端整車出貨。
因此，近期IC供應鏈的缺貨現象，已從消費性電子與電腦資通訊產業（Consumer & ICT），逐步蔓延到工控與車載市場（Industrial & Automotive）。過往車用半導體市場主要以垂直整合製造（IDM）或Fab-lite（即IDM企業將部分製造業務轉由協力廠商代工，是資產輕量化的一種策略）生產為主，例如恩智浦（NX

### 3. 隨機選出50個自由時報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 50;
```

In [33]:
ltn_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 100; """
ltn_test_urls = query_from_db(ltn_msn_sql)

In [31]:
ltn_sql_null = """
SELECT nrf.news_url 
FROM news_rss_feeds nrf
LEFT JOIN news_contents nc ON nrf.news_rss_feeds_id = nc.news_rss_feeds_id
WHERE nrf.rss_source = 'LTN' and nc.news_published_date IS NULL and nrf.processed_success = 1"""
ltn_test_urls = query_from_db(ltn_sql_null)

In [34]:
ltn_test_urls

,news_url
0,https://news.ltn.com.tw/news/life/breakingnews...
1,https://news.ltn.com.tw/news/life/paper/1443963
2,https://news.ltn.com.tw/news/politics/breaking...
3,https://sports.ltn.com.tw/news/breakingnews/34...
4,https://ent.ltn.com.tw/news/breakingnews/3428327
...,...
95,https://news.ltn.com.tw/news/society/breakingn...
96,https://news.ltn.com.tw/news/politics/breaking...
97,https://sports.ltn.com.tw/news/breakingnews/34...
98,https://news.ltn.com.tw/news/life/breakingnews...


In [37]:
def ltn_content_processor(url):
    postfix = ["""一手掌握經濟脈動
    點我訂閱自由財經Youtube頻道""", 
    """☆民眾如遇同居關係暴力情形，可撥打113保護專線，或向各地方政府家庭暴力防治中心求助☆""", 
    """☆健康新聞不漏接，按讚追蹤粉絲頁☆更多重要醫藥新聞訊息，請上自由健康網""",
    """《自由開講》是一個提供民眾對話的電子論壇，不論是對政治、經濟或社會、文化等新聞議題，有意見想表達、有話不吐不快，都歡迎你熱烈投稿。文長700字內為優，來稿請附真實姓名（必寫。有筆名請另註）、職業、聯絡電話、E-mail帳號。本報有錄取及刪修權，不付稿酬；請勿一稿多投，錄用與否將不另行通知。投稿信箱：LTNTALK@gmail.com""",
    """「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。""",
    """"""]
    res_dict = {}
    #headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
        else:
            try:
                res_dict['news_title'], res_dict['news_category'] = title_tag.string.split('|')
            except:
                pass
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag_1 = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('meta', attrs = {'name': 'pubdate'})
    time_tag_3 = soup.find('span', attrs = {'class': 'time'})
    if time_tag_1:
        try:
            d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%SZ")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%SZ")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))
    elif time_tag_3:
        try:
            d1 = datetime.datetime.strptime(time_tag_3.get_text(), "%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_3.get_text(), "%Y-%m-%d %H:%M:%S")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))
                

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag2 = soup.find('div', attrs = {'class':'text boxTitle boxText'})
    article_body_tag3 = soup.find('div', attrs = {'class':'text'})
    temp_content = []
    links = []
    links_descs = []

    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag2:
        p_tags = article_body_tag2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag2.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag3:
        p_tags = article_body_tag3.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag3.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        print('LTN {}, did not process properly'.format(url))
        #content_parser.logger.error('LTN url: {} did not process properly'.format(url))
        return
    return res_dict

In [44]:
print('自由時報')
print_util(ltn_test_urls[:10].values,ltn_content_processor)

自由時報

News Url: 
https://news.ltn.com.tw/news/life/breakingnews/3442519

News_title: 櫻花季吸客  清境農場多項優惠招睞親子客
News_category: 生活
News_fb_app_id: 140490219413038
News_keywords: 清境農場,賞櫻,清境,櫻花
News_description: 春節連假結束，高中職以下學校下週一開學，清境地區的山櫻花盛開，清境農場把握寒假最後兩天假期，祭出60週年場慶多項優惠，包括住宿7折、農特產品8成、餐廳9折，希望能吸引家長帶孩子上山賞花、看綿羊。溝境農場人員表示，山櫻花、富士櫻、吉野櫻接力開，櫻花魅力讓二二八連假訂房率已達8成。清境地區以草原、綿羊、藍天翠嶺聞名，這幾年則加入櫻花元素，櫻花群最多的在青青草原和五五街，目前是山櫻花（緋寒櫻）盛開期，3月開始則由富士櫻、吉野櫻接力開花，清境地區在二二八連假的訂房率已達8成。
Published_time: 2021-02-18 10:19:00
Published_date: 2021-02-18
News_related_url: []
News_related_url_desc: []
News: 清境地區的青青草原櫻花盛開，櫻花季可持續到3月下旬，櫻花魅力大，清境地區二二八連假訂房率已達8成。（清境農場提供）
〔記者陳鳳麗／南投報導〕春節連假結束，高中職以下學校下週一開學，清境地區的山櫻花盛開，清境農場把握寒假最後兩天假期，祭出60週年場慶多項優惠，包括住宿7折、農特產品8成、餐廳9折，希望能吸引家長帶孩子上山賞花、看綿羊。溝境農場人員表示，山櫻花、富士櫻、吉野櫻接力開，櫻花魅力讓二二八連假訂房率已達8成。
清境地區以草原、綿羊、藍天翠嶺聞名，這幾年則加入櫻花元素，櫻花群最多的在青青草原和五五街，目前是山櫻花（緋寒櫻）盛開期，3月開始則由富士櫻、吉野櫻接力開花，清境地區在二二八連假的訂房率已達8成。
高中職以下各級學校將在下週一開學，本週六、日是寒假最後兩天，但因週六（20日）是補班日，清境地區旅宿業者仍卯足勁宣傳，像經營青青草原和清境國民賓館的退輔會清境農場，即以農場60週年場慶的名義，20日推出多項優惠，包括住宿7折，餐廳用餐9折，購買農特產品8折。另外在青青草原

### 4. 隨機選出50個大紀元的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 50;
```

In [45]:
epoch_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 100; """
epoch_test_urls = query_from_db(epoch_test_sql)

In [48]:
def epoch_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ', 1)
        res_dict['news_title'] = title_category[0]
        
    
    category_tag = soup.find('meta', attrs = {'property':'article:section'})
    if category_tag:
        res_dict['news_category'] = str(category_tag['content'])
        
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Epoch date error {}'.format(e2))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'art-content'})
    #print(article_body_tag)
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag_2:
        p_tags = article_body_tag_2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag_2.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Epoch url: {} did not process properly'.format(url))

    return res_dict


In [49]:
print('大紀元：')
print_util(epoch_test_urls.values, epoch_content_processor)

大紀元：

News Url: 
https://www.epochtimes.com/b5/20/12/10/n12608555.htm

News_title: 歐冠16強出爐 皇馬後來居上 巴薩不敵尤文
News_category: 各地分網
News_fb_app_id: 1571885662850711
News_fb_page: 156786811025453
News_keywords: 16強,尤文,巴薩,曼聯,歐冠,皇馬,大紀元
News_description: 當地時間12月9日，2020/21賽季歐冠聯賽16強出爐。德甲和西甲球隊全都順利晉級。在「死亡之組」，皇馬主場2：0戰勝門興，兩隊攜手出線，國米和礦工出局；曼聯則在主場2：3不敵萊比錫，慘遭淘汰，大巴黎和萊比錫晉級。
Published_time: 2020-12-10 02:29:48
Published_date: 2020-12-10
News_related_url: ['https://www.epochtimes.com/b5/tag/%e6%ad%90%e5%86%a0.html', 'https://www.epochtimes.com/b5/tag/16%e5%bc%b7.html', 'https://www.epochtimes.com/b5/tag/%e7%9a%87%e9%a6%ac.html', 'https://www.epochtimes.com/b5/tag/%e7%9a%87%e9%a6%ac.html', 'https://www.epochtimes.com/b5/tag/%e5%b0%a4%e6%96%87.html', 'https://www.epochtimes.com/b5/tag/%e5%b7%b4%e8%96%a9.html', 'https://www.epochtimes.com/b5/tag/16%e5%bc%b7.html', 'https://www.epochtimes.com/b5/tag/%e5%b7%b4%e8%96%a9.html', 'https://www.epochtimes.com/b5/tag/%e6%ad%90%e5%86%a0.html', 'https://www.epochtimes.com/b5

### 5. 隨機選出50個經濟日報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 50;
```

In [50]:
udn_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 100; """
udn_test_urls = query_from_db(udn_test_sql)

In [52]:
import re
def udn_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category_lst = title_tag.string.split(' | ', 3)
        res_dict['news_title'] = title_category_lst[0]
        try:
            res_dict['news_category'] = title_category_lst[2]
        except:
            pass
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'shareBar__info--author'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.find('span').text, "%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d_temp = re.search('(\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2})', time_tag.text).group(0)
                d1 = datetime.datetime.strptime(d_temp, "%Y-%m-%d %H:%M") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                try:
                    d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%d %H:%M:%S") 
                    d2 = d1.date()
                    d1 -= datetime.timedelta(hours=8)
                    db_time_format = '%Y-%m-%d %H:%M:%S'
                    time_res = d1.strftime(db_time_format)
                    res_dict['published_time'] = time_res
                    
                    db_date_format = '%Y-%m-%d'
                    date_res = d2.strftime(db_date_format)
                    res_dict['published_date'] = date_res
                except Exception as e3:
                    print(e3)
                    content_parser.logger.info('Epoch date error {}'.format(e3))
    article_body_tag = soup.find('div', attrs = {'id':'article_body'})
    content_temp, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.get_text().strip() and p.get_text().strip() != 'facebook':
                    content_temp.append(p.get_text().strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs      
    content = '\n'.join(content_temp).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('udn url: {} did not process properly'.format(url))

    return res_dict


In [53]:
print_util(udn_test_urls.values, udn_content_processor)


News Url: 
https://money.udn.com/money/story/7307/5313501

News_title: 視察望海巷景觀跨橋 總統：盡覽海景將成熱門景點
News_category: 要聞
News_fb_app_id: 267846550050865
News_fb_page: 179043295447791
News_keywords: 林右昌,觀光,科技
News_description: 總統蔡英文今天到基隆視察望海巷漁港串聯計劃工程時說，望海巷景觀跨橋將銜接新北、基隆自行車道，加上可欣賞海景，相信未來絕對...
Published_time: 2021-03-12 07:21:00
Published_date: 2021-03-12
News: 總統蔡英文今天到基隆視察望海巷漁港串聯計劃工程時說，望海巷景觀跨橋將銜接新北、基隆自行車道，加上可欣賞海景，相信未來絕對會成為熱門打卡景點。
蔡總統上午在基隆市長林右昌陪同下，前往望海巷漁港，視察望海巷漁港串聯計劃工程。林右昌也向蔡總統報告，他將在今天晚間與一群偉士牌車友們騎機車從基隆出發，挑戰台灣縱走24小時不斷電「切西瓜」行程。蔡總統特別在他所騎乘的機車上簽名「加持」，祝福行程順利成功。
蔡總統致詞時說，八斗子有山、有海、有港口、有漁村文化，也有小朋友最喜歡的國立海洋科技博物館。望海巷景觀跨橋工程設計由親民黨主席宋楚瑜的女兒宋鎮邁操刀，外型漂亮，相信未來將是熱門打卡的景點。
她認為，林右昌擔任基隆市長多年來，大家可見證到基隆有很大的改變，主要是他有城市規劃的專業，以有系統、有遠見的規劃，讓各界都可感受到基隆這幾年來的改變。望海巷漁港串聯計劃工程就是用前瞻基礎建設計畫支援，這部分也要感謝立委蔡適應過去在立院捍衛這些預算。
蔡總統表示，林右昌今晚要挑戰從基隆到墾丁騎機車「台灣切西瓜」的行程，他除要順道宣傳10月在基隆市舉辦的城市博覽會，也要把基隆的雨水一路帶到中南部，他的使命值得稱許，也很重要，祝福他行程圓滿順利，讓更多人看到基隆的不一樣。
蔡總統隨後到基隆天顯宮參拜祈福時說，去年台灣的經濟成長率為3.11%，是先進國家中最高的，也是亞洲四小龍之首，相信未來幾年台灣的經濟都會在健康狀態下持續成長，但現在疫情還未遠離，且還有旱災等許多挑戰，呼籲國人持續團結努力，讓世界各地的台灣

### 6. 隨機選出50個新浪台灣新聞中心的新聞連結： (skip this phase, too many error urls)
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 50;
```

In [54]:
sina_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 20; """
sina_test_urls = query_from_db(sina_test_sql)

In [27]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [ ]:
import requests
from itertools import cycle
import traceback
#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [ ]:
import random
# This data was created by using the curl method explained above
headers_list = [
    # Firefox 77 Mac
     {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
        "Connection": "keep-alive",
        "DNT": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
        # Chrome 83 Windows 
    {
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9"
    }
]

def sina_content_processor(url):
    res_dict = {}
   # proxy = next(proxy_pool)
    r = requests.get(url, headers = random.choice(headers_list))
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-1].strip()
    
    
    # Sina did not have FB available
    #fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    #if fb_app_tag:
    #    res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']
        
    time_tag = soup.find('meta', attrs = {'name': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Sina date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'id':'article_content'})
    article_body_tag_2 = soup.find('div', attrs = {'class': 'pcont'})
    if article_body_tag:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content
    elif article_body_tag_2:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Sina url: {} did not process properly'.format(url))

    return res_dict


In [ ]:
print_util(sina_test_urls.values, sina_content_processor)

### 7. 隨機選出50個PCHOME的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 50;
```

In [62]:
pchome_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 20; """
pchome_test_urls = query_from_db(pchome_test_sql)

In [63]:

def pchome_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-2].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%S") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%SZ")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('PChome date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'calss':'article_text'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        a_tags = article_body_tag.find_all('a')
        if content:
            content = re.sub('(\n)+', '\n', content)
            content = re.sub(r'(相關新聞[\s\S]+)', '', content)
            res_dict['news'] = content 
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PChome url: {} did not process properly'.format(url))

    return res_dict

In [64]:
print_util(pchome_test_urls.values, pchome_content_processor)


News Url: 
https://news.pchome.com.tw/politics/taiwanhot/20210324/index-61657715571312221001.html

News_title: 立院朝野協商通過成立疫苗採購調閱小組 陳時中：尊重
News_category: 政治新聞
News_fb_app_id: 498546706899526
News_fb_page: 1423378217880597
News_keywords: 中國國民黨,口罩,台灣,國民黨,旅遊,武漢肺炎,民眾黨,疫情指揮中心,疫苗,防疫,陳時中
News_description: 政治中心／綜合報導立法院衛環委員會3月18日由中國國民黨立委提案、民眾黨團表決通過成立「武漢肺炎疫苗採購調閱專案小組」，擬進一步了解台灣向外國藥廠的武漢肺炎疫苗價格，其相關議題近幾日引起爭論。…
Published_time: 2021-03-24 09:12:35
Published_date: 2021-03-24
News: 政治中心／綜合報導
立法院衛環委員會3月18日由中國國民黨立委提案、民眾黨團表決通過成立「武漢肺炎疫苗採購調閱專案小組」，擬進一步了解台灣向外國藥廠的武漢肺炎疫苗價格，其相關議題近幾日引起爭論。今（24）日立院衛環委員會經朝野協商後，同意照案通過，至於調閱小組成立時間交由朝野協商之後決定。
近期外界除關切我國向外國採買的武肺疫苗，其抵台時程外，亦頻探詢採買價格。中央流行疫情指揮中心則一再表示，武肺疫苗配送時程、疫苗採購價格目前是賣方市場，涉及合約保密條款無法透露，若公開價格會影響台灣取得進口武肺疫苗，非全民之福。去（2020）年底亦曾傳出比利時官員洩露各廠武肺疫苗採購價，引發輝瑞＆BNT公司強烈不滿。
對於立院衛環委員會經朝野協商後，經表決通過成立「新冠疫苗採購調閱小組」。國民黨立院黨團表示，成立調閱小組的目的是要釐清真相，不讓全民成為疫苗採購的冤大頭，為人民守著每一分血汗錢。
針對立院相關決議，中央流行疫情指揮中心指揮官陳時中，於今（24）日下午在武肺疫情記者會上回應，對此結果予以尊重。
※台灣好新聞提醒您：武漢肺炎防疫不恐慌！
肥皂勤洗手、公共場所戴口罩，減少聚會不握手，避免接觸禽畜類動物、生食肉品！
國外入境若身體不適主動通報機場及港口檢疫人員，

### 8. 隨機選出50個Ettoday的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 50;
```

In [65]:
ettoday_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 20; """
ettoday_test_urls = query_from_db(ettoday_test_sql)

In [69]:
def ettoday_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    # Ettoday did not put fb page tag on the content
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    #if fb_page_tag:
        #res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('time', attrs = {'itemprop': 'datePublished'}) 
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get_text().strip(), "%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e2:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get_text().strip(), "%Y年%m月%d日 %H:%M")
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e3:
                print(e3)
                content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))

    

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    temp_content = []
    links, links_descs = [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        
        if p_tags:
            for p in p_tags:
                # Ignore the image caption
                if p.find('strong'):
                    for match in p.find_all('strong', attrs = {'class': 'figcaption'}):
                        match.decompose()
                #    continue
                if p.text:
                    
                    temp_content.append(p.text.strip())
                print(p.text)
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip().replace('\u3000', ' '))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Ettoday url: {} did not process properly'.format(url))

    return res_dict

In [70]:
print_util(ettoday_test_urls.values, ettoday_content_processor)



記者周康玉／台北報導
因應台中地區缺水嚴峻，用水大戶半導體廠台積電今（25）日上午10點於台中勤美之森建案取水。
台積電表示，日前多位建商包含勤美集團、成中恆營造、富旺國際、惠宇建設、寶輝建設及大松建設等，在台灣水情緊張之際皆主動與台積電聯繫以避免流失地下水資源。台積電秉持珍惜用水的信念積極評估，並即時投入資源安排取水、載水等相關作業，希望在符合標準的最大限度內，透過有效利用水資源，減少水庫用水以舒緩部分水情吃緊現象。

▲台中地區建商將新建案動土、開挖期間的排放地下水，提供台積電回收再利用。（圖／記者鄧木卿攝）
旱災中央災害應變中心昨（24）日決議台中、苗栗地區轉為紅燈，將自4月6日起分區供五停二，節水率將由11%提升為15%，其中工業限水措施更提早，自4/1起節水率由11%提升至13%。

▲台中地區建商將新建案動土、開挖期間的排放地下水，提供台積電回收再利用。（圖／記者鄧木卿攝）

就各限水階段備有因應計畫，因應政府相關單位進一步限水政策，部分廠區將微幅提升水車載水量以提高用水彈性，並不會影響營運。
News Url: 
http://feedproxy.google.com/~r/ettoday/finance/~3/f5Im2kda6hY/1946042.htm

News_title: 中部水情亮紅燈！　台積電取台中勤美新建案地下水
News_category: ETtoday財經雲
News_fb_app_id: 146858218737386
News_keywords: 台積電,缺水,影音
News_description: 台中地區缺水嚴峻，用水大戶半導體廠台積電今（25）日表示，已經有些台中地區的建商向台積電「伸出援手」，欲將新建案動土、開挖期間的排放地下水，提供台積電回收再利用。據了解，台積電表示確實有不只一家建商提供意見，經評估後，已在今日上午10點於台中勤美之森建案取水。(台積電,缺水,影音)
Published_time: 2021-03-25 03:20:00
Published_date: 2021-03-25
News: 記者周康玉／台北報導
因應台中地區缺水嚴峻，用水大戶半導體廠台積電今（25）日上午10點於台中勤美之森建案取水。
台積電表示，日前多位建商包含勤美集團、成中恆營造、富旺國際、惠宇建設、寶輝建設及大松建設等，在

### 9. 隨機選出50個Rti 中央廣播電臺的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 50;
```

In [71]:
rti_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 20; """
rti_test_urls = query_from_db(rti_test_sql)

In [75]:
def rti_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    reserved_keywords = ['Rti', '中央廣播電臺', 'Radio Taiwan International']
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ', 1)
        res_dict['news_title'] = title_category[0]
    category_tag = soup.find('div', attrs = {'class':'swiper-wrapper'})
    if category_tag:
        
        res_dict['news_category'] = category_tag.find('a', attrs = {'class':'active'}).get_text().strip()

    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        keyword_lst = [keyword for keyword in keywords_tag['content'].split(',') if keyword not in reserved_keywords]
        res_dict['news_keywords'] = ','.join(keyword_lst)

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if description_tags:
        res_dict['news_description'] = description_tags[1]['content']

    time_tag = soup.find('li', attrs = {'class': 'date'})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M:%S")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                #content_parser.logger.info('RTI url: {} date not process properly, error message {}'.format(url, e2))

    article_body_tag = soup.find('article', attrs = {'class' : None})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('RTI url: {} did not process properly'.format(url))
    return res_dict

In [76]:
print_util(rti_test_urls.values, rti_content_processor)


News Url: 
https://www.rti.org.tw/news/view/id/2090801

News_title: 直接找台灣半導體業者下單！福斯：車用晶片短缺問題年中可望解決
News_category: 國際
News_fb_app_id: 1722151021422152
News_keywords: 晶圓,晶片,車用晶片
News_description: 因晶片短缺被迫減產的德國福斯汽車（Volkswagen）主管透露，由於半導體廠承諾提高產量，今年下半年問題可望解決，專家研判汽車供應鏈的生態可能從此改變。德國「商報」（Handelsblatt）今(2)日報導，福斯一位不具名的主管透露，目前減產的汽車介於10萬到20萬輛，未來幾個月晶片短缺的問題仍將...
Published_time: 2021-02-02 12:57:00
Published_date: 2021-02-02
News: 因晶片短缺被迫減產的德國福斯汽車（Volkswagen）主管透露，由於半導體廠承諾提高產量，今年下半年問題可望解決，專家研判汽車供應鏈的生態可能從此改變。
德國「商報」（Handelsblatt）今(2)日報導，福斯一位不具名的主管透露，目前減產的汽車介於10萬到20萬輛，未來幾個月晶片短缺的問題仍將不時出現，不過由於半導體廠承諾在年中大幅提高車用晶片的產量，因此問題之後可望顯著改善。
福斯職工委員會主席歐斯特羅（Bernd Osterloh）也預期，上半年仍有晶片荒的問題，不過隨著晶片供應量增加和疫情緩和，下半年汽車產量可望增加，屆時員工可能連週末都要加班，才能補足上半年短少的產量。
業界盛傳，為解決晶片供應短缺的問題，福斯繞過長期合作的博世（Bosch）等汽車零件業者和英飛凌（Infineon）等車用晶片業者，直接找台灣的半導體業者下單，不過福斯沒有透露合作的台灣廠商。
英飛凌、恩智浦（NXP）、德州儀器（Texas Instruments）、意法半導體（STMicroelectronics）等主要車用晶片業者無法滿足汽車工業的需求，福斯被迫直接找半導體業者下單，長期來看可能改變汽車供應鏈的生態。
資誠會計師事務所（PwC）半導體業專家格羅格爾（Marcus Gloger）研判，晶片在汽車的重要性持續提升，自己設計的晶片和軟體效能更好，因此汽

### 10. 隨機選出50個公視新聞網的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 50;
```

In [77]:
pts_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 20; """
pts_test_urls = query_from_db(pts_test_sql)

In [79]:
def pts_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit(' | ', 1)
        res_dict['news_title'] = title_category[0].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if len(description_tags) > 1:
        res_dict['news_description'] = description_tags[1]['content']
        
    time_tag_2 = soup.find('div', attrs = {'class': 'maintype-wapper hidden-lg hidden-md'})
    time_tag = soup.find('time', attrs = {'class': None})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get_text(), "%Y-%m-%d %H:%M") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get_text(), "%Y-%m-%d %H:%M:%S") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
            
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        date_tag = time_tag.find('h2')
        if date_tag:
            try:
                d1 = datetime.datetime.strptime(date_tag.text, "%Y年%m月%d日")
                d2 = d1 + datetime.timedelta(hours=8)
                d2 = d2.date()
                #d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e1:
                print(e1)
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e1, url))
    

    article_body_tag = soup.find('div', attrs = {'class':'article_content'})
    article_body_tag_2 = soup.find('article', attrs = {'class': 'post-article'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        if content:
            res_dict['news'] = content
    elif article_body_tag_2:
        content = article_body_tag_2.text.strip()
        if content:
            res_dict['news'] = content
    
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PTS url: {} did not process properly'.format(url))

    return res_dict

In [83]:
print_util(pts_test_urls.values, pts_content_processor)


News Url: 
https://news.pts.org.tw/article/513888

News_title: 新冠病毒不斷突變 比利時研發單一疫苗 ｜ 公視新聞網 PNN
News_fb_app_id: 466487386878484
Published_time: 2021-02-20 04:31:00
Published_date: 2021-02-20
News: 由於新型冠狀病毒出現不同的變種，提升了散播的速度與防疫的難度。美國財星雜誌的報導指出，比利時就有新創企業，正試圖找出足以對付所有變種病毒的單一疫苗，以求一勞永逸。不過美國休斯醫療研究所認為，無論是甚麼病毒，總在傳染途中發生突變，因此現階段首要之務，還是要設法阻止新增病例的發生。
 
根據各國最新公佈的數據，全球單日新增的新冠病毒確診人數，已經從今年1月7日的高峰84萬多人，持續下降到2月18日的40萬左右。基於這樣的理由，各國紛紛有地方政府要求中央主管機關，同意鬆綁相關的防疫管制措施，但衛生官員對此態度相當保留。
德國衛生部長史巴恩說：「各地要求解除管制的呼聲不絕於耳，但我們對於鬆綁必須非常非常小心。畢竟要達到現在的成果殊屬不易。」
除了防疫管制，疫苗施打也正如火如荼進行中。不過新冠病毒的疫苗，現階段只能做到針對單一蛋白質結構，也就是一種疫苗對付一種病毒。然而已知的變種病毒先後在英國、南非與巴西出現，讓防疫人員焦頭爛額。
美國財星雜誌最新的報導指出，有研究人員打算仿效流感疫苗的模式，研發所謂多價疫苗，也有藥廠走另一個方向，試圖找出新冠病毒的核心基因，設法做出足以對付所有變種病毒的單一疫苗。然而，美國的休斯醫療研究所，則呼籲各方回歸本質，了解病毒的特性。
美國休斯醫療研究所衛教宣導影片：「病毒這種東西不可能靠自己存活，而必須仰賴傳染給人類（才能存活）。病毒要入侵並佔領人體細胞才能複製，每一次新的感染發生就讓病毒有機會產生突變。研究顯示現有疫苗仍可對抗變種病毒，但有些疫苗的效果不是那麼好。防範病毒突變的最佳途徑，就是阻止新的感染病例發生。」
專家認為，在疫苗不一定百分之百有效的情況下，採取必要的防疫管制措施，又要安撫民眾波動的情緒，是各國政府必須面對的課題。



News Url: 
https://news.pts.org.tw/article/521521

News_title

### 11. 隨機選出50個新頭殼要聞的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 50;
```

In [84]:
newstalk_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 20; """
newstalk_test_urls = query_from_db(newstalk_test_sql)

In [67]:
def newstalk_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('NewsTalk date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    #print(article_body_tag)
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')#, attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('NewsTalk url: {} did not process properly'.format(url))

    return res_dict

In [68]:
print_util(newstalk_test_urls.values, newstalk_content_processor)


News Url: 
https://newtalk.tw/news/view/2020-12-22/512583

Title: 去職震撼彈 林文淵：今天是我在大同最後一天
News: 大同董事長林文淵在11月接掌大同後，並在今天親上火線說明營運表現。林文淵卻爆出震撼彈，他指出，「今天是我在大同最後一天」。
百年大同公司經營權易主，林文淵在11月接任大同董事長，大同在今天召開記者會，林文淵親自上陣，先說明他到大同50天後所做的努力與改變。
不過，林文淵指出，大股東上週通知他將更換法人代表，原因是「時空環境不同」，雖然不懂更改原因，但他只能接受，將結束手邊工作。



News Url: 
https://newtalk.tw/news/view/2021-01-11/521894

Title: 明日開戰戴資穎直呼緊張找男雙對練
News: 超級1000系列泰國公開賽明日開戰，我國世界球后戴資穎睽違近10個月重返國際賽，今(11)日發文透露「感覺有點緊張」，也PO出到比賽場地訓練的影片，並找來我國第一男雙「麟洋配」王齊麟／李洋對打適應場地。
小戴去年3月15日(倫敦當地時間)擊敗中國一姊陳羽菲，抱回全英公開賽冠軍盃，並重返球后寶座，之後因新冠肺炎全球肆虐，國際賽開始停擺，去年10月的丹麥賽，小戴考慮疫情放棄衛冕，因此明天登場的泰國賽將是疫情後小戴的首戰。
小戴今天發文透露心情，「是不是真的很久沒有看我比賽了？ 我也一樣很久沒比賽了，感覺有點緊張。但不管怎麼樣，都希望可以享受回到場上的感覺。明天就有賽程囉，不要忘了看我打球。」
小戴也請來本站同樣要出賽的「麟洋配」對打，適應場地，「今天來到了主場訓練，感謝兩位的協助。明天要比賽啦」小戴首戰對手將是地主18歲小將Benyapa Aimsaard，雙方尚未有對戰紀錄。

延伸閱讀： 美軍方警告 拜登就職日須防範進一步威脅 NBA／表弟擊中詹皇頭部 惡意犯規遭驅逐出場 NBA／厄文因個人因素連缺3場 杜蘭特：我們支持他



News Url: 
https://newtalk.tw/news/view/2021-01-07/520004

Title: 「昨天有去探望鍾沛君」 柯文哲：PM2.5要嚴肅面對
News: 台北市國民黨籍議員鍾沛君日前檢查出右肺異常，經過開刀手術順利成功，市長柯文哲今(7)下午透露昨天有去探

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import jieba
import numpy as np
# 如果檔案內有一些編碼錯誤，使用 errors='ignore' 來忽略錯誤
with open("107.csv", encoding="big5", errors='ignore') as f:
    text = f.read()

# 設定使用 big5 斷詞
jieba.set_dictionary('dict.txt.big')
wordlist = jieba.cut(text)
words = " ".join(wordlist)
#文字雲造型圖片
mask = np.array(Image.open('picture.png')) #文字雲形狀
# 從 Google 下載的中文字型
font = 'SourceHanSansTW-Regular.otf'
#背景顏色預設黑色，改為白色、使用指定圖形、使用指定字體
my_wordcloud = WordCloud(background_color='white',mask=mask,font_path=font).generate(words)

plt.imshow(my_wordcloud)
plt.axis("off")
plt.show()
#存檔
wordcloud.to_file('word_cloud.png')

In [33]:
import numpy as np
np.log(100)


4.605170185988092

In [35]:
import sys
sys.version

'3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'